In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

# Steps
1. fill missing values
2. combine important features
3. convert them into vectors using TfidVectorizer
4. use cosine similarity to establish similarity
5. take user input
6. locate the movie in dataframe movies,get its index
7. evaluate the similarity score using the index
8. sort it and display

In [2]:
df=pd.read_csv("data/movies.csv")
df.head()


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [4]:
df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [5]:
df1=df.copy()


In [6]:
selected_features=["genres","keywords","production_companies","cast","crew","director","tagline"]
for feature in selected_features:
    df1[feature]=df1[feature].fillna("")
    

In [7]:
df1.isna().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [9]:
combinedtext=df1["genres"]+df1["keywords"]+df1["production_companies"]+df1["cast"]+df1["crew"]+df1["director"]+df1["tagline"]
combinedtext
# ["genres","keywords","production_companies","cast","crew","director","tagline"]

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy Romance[]Edward Burns Kerry Bish\u00e9 ...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    []Daniel Henney Eliza Coupe Bill Paxton Alan R...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [14]:
vectorizer=TfidfVectorizer(stop_words="english")
featurevec=vectorizer.fit_transform(combinedtext)
print(featurevec)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 781120 stored elements and shape (4803, 238849)>
  Coords	Values
  (0, 186550)	0.0053763605968131
  (0, 186764)	0.006238381604871452
  (0, 201933)	0.008059494890484058
  (0, 227724)	0.006820109279950548
  (0, 202527)	0.01882759037329202
  (0, 195178)	0.014656761429600696
  (0, 203846)	0.010842005844025329
  (0, 229906)	0.02266647744362858
  (0, 236503)	0.008723634538258135
  (0, 195610)	0.016142440675541707
  (0, 229597)	0.013787694782625521
  (0, 209177)	0.012516532628307087
  (0, 202737)	0.010963486505894491
  (0, 221760)	0.011365962747993508
  (0, 208894)	0.33819635402978654
  (0, 41569)	0.013023204099234334
  (0, 233964)	0.008648089024063976
  (0, 194200)	0.008487259951109145
  (0, 203353)	0.007018866138209068
  (0, 196407)	0.007455256490475707
  (0, 41956)	0.00870449707111766
  (0, 200099)	0.010668458206318152
  (0, 201071)	0.00997989811991577
  (0, 44583)	0.011537524853293324
  (0, 214256)	0.016472844480375833
  :	:
  

In [15]:
similarity=cosine_similarity(featurevec)

In [16]:
similarity

array([[1.        , 0.53910165, 0.80572135, ..., 0.33059433, 0.0845941 ,
        0.18307775],
       [0.53910165, 1.        , 0.54465701, ..., 0.29380643, 0.07253634,
        0.16057338],
       [0.80572135, 0.54465701, 1.        , ..., 0.34821872, 0.09577578,
        0.18637772],
       ...,
       [0.33059433, 0.29380643, 0.34821872, ..., 1.        , 0.04928183,
        0.11619233],
       [0.0845941 , 0.07253634, 0.09577578, ..., 0.04928183, 1.        ,
        0.02893122],
       [0.18307775, 0.16057338, 0.18637772, ..., 0.11619233, 0.02893122,
        1.        ]], shape=(4803, 4803))

TypeError: 'RangeIndex' object is not callable

In [18]:
movies=df1["title"].tolist()

In [19]:
movies

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [21]:
movieuser=input("enter your favourite movie of all time")
moviedf=difflib.get_close_matches(movieuser,movies)[0]
moviedf

enter your favourite movie of all time iron man


'Iron Man'

In [30]:
index=df[df["title"]==moviedf].index[0]

In [31]:
index

np.int64(68)

In [54]:
similarity_score=list(enumerate(similarity[index]))
top10=sorted_movies=sorted(similarity_score,key=lambda x : x[1],reverse=True)[1:11]

In [55]:
for item in top10:
    print(movies[item[0]])

Iron Man 2
Ant-Man
Captain America: Civil War
Hulk
Iron Man 3
Blade: Trinity
Spider-Man
Avengers: Age of Ultron
Spider-Man 2
Terminator 3: Rise of the Machines


In [60]:
## all in one cell
movie_input=input("enter your favourite movie of all time")
movie_df=difflib.get_close_matches(movie_input,movies)[0]
index=df1[df1["title"]==movie_df].index[0]
score=list(enumerate(similarity[index]))
sorted_score=sorted(score,key=lambda x : x[1] ,reverse=True)[1:11]
print(movie_df,"if this is incorrect we dont have it in our dataframe")
print("Top 10 Recommendations")
for item in sorted_score:
    print(movies[item[0]])


enter your favourite movie of all time superman


Superman if this is incorrect we dont have it in our dataframe
Top 10 Recommendations
Superman II
Bad Boys II
Superman III
Catwoman
Terminator 3: Rise of the Machines
Batman v Superman: Dawn of Justice
Minority Report
Alexander
Batman Begins
The Bourne Supremacy
